# 과태료 데이터 전처리

## 라이브러리 및 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import requests
import json

In [2]:
PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data/'
# PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data/'

In [3]:
df = pd.read_csv(PATH + "02_과태료/흡연과태료 부과 내역서.csv", encoding ='cp949')
df

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세
0,2012-000002-00,"대전광역시 중구 동서대로11길 6,202호 (태평동,나래주택)",2012-07-03,2012-07-18,"40,000","40,000"
1,2014-000001-00,"302-841 대전광역시 서구 둔산로 223 , 6동 907호 （둔산동, 청솔아파트）",2014-08-12,2014-08-27,"80,000","80,000"
2,2014-000002-00,"302-831 대전광역시 서구 둔산북로 160 , 102동 1401호 （둔산동, ...",2014-08-12,2014-08-27,"80,000","80,000"
3,2014-000003-00,"302-861 대전광역시 서구 유등로669번길 40 , 101호 （탄방동, 아트빌）",2014-08-12,2014-08-27,"80,000","80,000"
4,2014-000004-00,"302-831 대전광역시 서구 둔산북로 175 , 7동 1404호 （둔산동, 햇님...",2014-08-12,2014-08-27,"80,000","80,000"
...,...,...,...,...,...,...
3710,2021-000024-00,금연구역 내 흡연행위,2021-06-03,2021-06-23,"80,000","80,000"
3711,2021-000025-00,금연구역 내 흡연행위,2021-06-03,2021-06-23,"80,000","80,000"
3712,2021-000027-00,금연구역 내 흡연행위,2021-07-05,2021-07-31,"50,000","50,000"
3713,2021-000028-00,급연구역 내 흡연행위,2021-07-19,2021-08-09,"24,000","24,000"


## data check

In [4]:
# 결측치
df.isnull().sum()

부과번호       0
부과대상      72
부과일자       0
최종납기일자     0
최초본세       0
최종본세       0
dtype: int64

In [5]:
df[df['부과대상'].isnull()]

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세
2264,2019-000190-00,NaN,2019-08-26,2019-09-16,"24,000","24,000"
2265,2019-000191-00,NaN,2019-08-24,2019-09-16,"24,000","24,000"
2538,2015-000007-00,NaN,2015-01-26,2015-02-10,"80,000","80,000"
2687,2014-000003-00,NaN,2014-05-15,2014-06-03,"80,000","80,000"
2930,2019-000022-00,NaN,2019-07-02,2021-08-31,"100,000","100,000"
...,...,...,...,...,...,...
3534,2021-000035-00,NaN,2021-05-27,2021-06-11,"24,000","24,000"
3535,2021-000036-00,NaN,2021-05-27,2021-06-11,"24,000","24,000"
3536,2021-000037-00,NaN,2021-05-27,2021-06-11,"24,000","24,000"
3537,2021-000038-00,NaN,2021-06-14,2021-08-17,"30,000","30,000"


부과대상에 있는 주소 정보를 기준으로 위치정보(위도, 경도, 행정동)을 추출해야함  
부과대상에 정보가 없는 경우 진행 불가능하여 drop 결정

In [6]:
df = df.dropna(subset=['부과대상'])
df.isnull().sum()

부과번호      0
부과대상      0
부과일자      0
최종납기일자    0
최초본세      0
최종본세      0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3643 entries, 0 to 3714
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   부과번호    3643 non-null   object
 1   부과대상    3643 non-null   object
 2   부과일자    3643 non-null   object
 3   최종납기일자  3643 non-null   object
 4   최초본세    3643 non-null   object
 5   최종본세    3643 non-null   object
dtypes: object(6)
memory usage: 199.2+ KB


최초본세와 최종본세는 금액으로 `int` 타입으로 변경  
타입변경을 위해 `,`를 삭제해야함

## 최초본세 수정

In [8]:
df['최초본세'] = df['최초본세'].str.replace(pat=',', repl='')

## 부과대상 형태 확인

In [9]:
df['부과대상'].str[:5].unique()

array(['대전광역시', '302-8', '306-8', ..., '금연구역내', '407-7', '급연구역 '],
      dtype=object)

대전광역시 외에 다른 형태의 잡음들이 확인된다.

### 대전광역시 데이터만 추출

In [10]:
df_dj = df[df['부과대상'].str.contains('대전')]
df_dj

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세
0,2012-000002-00,"대전광역시 중구 동서대로11길 6,202호 (태평동,나래주택)",2012-07-03,2012-07-18,40000,"40,000"
1,2014-000001-00,"302-841 대전광역시 서구 둔산로 223 , 6동 907호 （둔산동, 청솔아파트）",2014-08-12,2014-08-27,80000,"80,000"
2,2014-000002-00,"302-831 대전광역시 서구 둔산북로 160 , 102동 1401호 （둔산동, ...",2014-08-12,2014-08-27,80000,"80,000"
3,2014-000003-00,"302-861 대전광역시 서구 유등로669번길 40 , 101호 （탄방동, 아트빌）",2014-08-12,2014-08-27,80000,"80,000"
4,2014-000004-00,"302-831 대전광역시 서구 둔산북로 175 , 7동 1404호 （둔산동, 햇님...",2014-08-12,2014-08-27,80000,"80,000"
...,...,...,...,...,...,...
3565,2014-000031-00,300-834 대전광역시 동구 중심2길 26 （판암동）,2014-08-21,2014-09-11,80000,"80,000"
3569,2014-000039-00,"300-826 대전광역시 동구 계족로424번길 19 , 2층동 （용전동）",2014-12-10,2014-12-30,80000,"80,000"
3570,2015-000001-00,"300-714 대전광역시 동구 은어송로 117 , 309동 303호 （가오동, 은...",2015-01-05,2015-02-02,100000,"100,000"
3571,2015-000002-00,대전광역시 동구 용전동 12번지 17호 101호,2015-01-05,2015-02-02,50000,"50,000"


In [11]:
df_dj['부과대상'].str[:5].unique()

array(['대전광역시', '302-8', '306-8', '302-7', '301-8', '305-3', '301-2',
       '305-7', '305-8', '301-7', '306-7', '300-7', '300-2', '300-8',
       '306-1', '302-9', '305-2', '35378', '35258', '35255', '35245',
       '35317', '35316', '35267', '34178', '34962', '35371', '35308',
       '35386', '35272', '35273', '35292', '34140', '35221', '35347',
       '35322', '35294', '34812', '35297', '35298', '35282', '35265',
       '35271', '35328', '34232', '35376', '35373', '35381', '35059',
       '34076', '35244', '34022', '35309', '35310', '34810', '35246',
       '35325', '35321', '35334', '35356', '35295', '35228', '34396',
       '35398', '34598', '35389', '35266', '34085', '35230', '35364',
       '35215', '35426', '35372', '35311', '35358', '34845', '35355',
       '35314', '34636', '35251', '35236', '35211', '35264', '35262',
       '35277', '34880', '35289', '34172', '34070', '35206', '35331',
       '35395', '34554', '35359', '35079', '34661', '35303', '35330',
       '35300', '353

#### 두가지 형태 존재
* `대전광역시` : 문제 없음
* `지번 or 도로명지번 + 대전광역시` : 앞의 지번을 삭제해줘야함

대전광역시를 한번 더 추가하여 split, 첫번째 공백으로 구분

In [12]:
df_split = df_dj.copy()
df_split['부과대상'] = df_dj["부과대상"].str.replace(pat='대전광역시', repl='대전광역시 대전광역시')

In [13]:
a = df_split['부과대상'].str.split(' ', expand=True, n=1)

In [14]:
a

,0,1
0,대전광역시,"대전광역시 중구 동서대로11길 6,202호 (태평동,나래주택)"
1,302-841,"대전광역시 대전광역시 서구 둔산로 223 , 6동 907호 （둔산동, 청솔아파트）"
2,302-831,"대전광역시 대전광역시 서구 둔산북로 160 , 102동 1401호 （둔산동, 한마..."
3,302-861,"대전광역시 대전광역시 서구 유등로669번길 40 , 101호 （탄방동, 아트빌）"
4,302-831,"대전광역시 대전광역시 서구 둔산북로 175 , 7동 1404호 （둔산동, 햇님아파트）"
...,...,...
3565,300-834,대전광역시 대전광역시 동구 중심2길 26 （판암동）
3569,300-826,"대전광역시 대전광역시 동구 계족로424번길 19 , 2층동 （용전동）"
3570,300-714,"대전광역시 대전광역시 동구 은어송로 117 , 309동 303호 （가오동, 은어송..."
3571,대전광역시,대전광역시 동구 용전동 12번지 17호 101호


In [15]:
df_split['부과대상'] = a[1]

In [16]:
# 대전광역시 제거
df_split['부과대상'] = df_split["부과대상"].str.replace(pat='대전광역시 ', repl='')
df_split

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세
0,2012-000002-00,"중구 동서대로11길 6,202호 (태평동,나래주택)",2012-07-03,2012-07-18,40000,"40,000"
1,2014-000001-00,"서구 둔산로 223 , 6동 907호 （둔산동, 청솔아파트）",2014-08-12,2014-08-27,80000,"80,000"
2,2014-000002-00,"서구 둔산북로 160 , 102동 1401호 （둔산동, 한마루아파트）",2014-08-12,2014-08-27,80000,"80,000"
3,2014-000003-00,"서구 유등로669번길 40 , 101호 （탄방동, 아트빌）",2014-08-12,2014-08-27,80000,"80,000"
4,2014-000004-00,"서구 둔산북로 175 , 7동 1404호 （둔산동, 햇님아파트）",2014-08-12,2014-08-27,80000,"80,000"
...,...,...,...,...,...,...
3565,2014-000031-00,동구 중심2길 26 （판암동）,2014-08-21,2014-09-11,80000,"80,000"
3569,2014-000039-00,"동구 계족로424번길 19 , 2층동 （용전동）",2014-12-10,2014-12-30,80000,"80,000"
3570,2015-000001-00,"동구 은어송로 117 , 309동 303호 （가오동, 은어송마을3단지）",2015-01-05,2015-02-02,100000,"100,000"
3571,2015-000002-00,동구 용전동 12번지 17호 101호,2015-01-05,2015-02-02,50000,"50,000"


In [17]:
df_split['부과대상'].str[:3].unique()

array(['중구 ', '서구 ', '대덕구', '유성구', '동구 ', ' 서구'], dtype=object)

#### 앞의 공백 및 중복 공백 제거

In [18]:
print(df_split['부과대상'].str.contains('  ').sum()) # 스페이스바 두번
print(df_split['부과대상'].str.contains('   ').sum()) # 스페이스바 세번

2761
0


In [19]:
df_split["부과대상"] = df_split["부과대상"].str.lstrip()
df_split["부과대상"] = df_split["부과대상"].str.replace(pat='  ', repl=r' ', regex=True)

### 도로명주소 정제

* 괄호와 괄호안 정보 제거
* `,` 를 기준으로 split

In [20]:
df_split["부과대상"] = df_split["부과대상"].str.replace(pat=r'\([^)]*\)', repl=r' ', regex=True)

In [21]:
# ()가 남아있는지 확인
df_split[df_split["부과대상"].str.contains('\)')]

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세
2273,2019-000199-00,중구 인창로 5 （대흥동）),2019-08-26,2019-09-16,24000,"24,000"
2274,2019-000200-00,"유성구 문지로 300 （문지동, 효성해링턴플레이스） 102동 1105호)",2019-08-26,2019-09-16,24000,"24,000"
2698,2014-000016-00,대덕구 대덕대로1447번길 40 대전공장) 내 뉴엠,2014-10-02,2015-04-30,100000,"100,000"
2935,2019-000027-00,"대덕구 아리랑로 211 , 106동 703호 （법동, 영진로얄아파트)",2019-07-01,2019-07-22,80000,"80,000"
2936,2019-000028-00,"대덕구 덕암로 200 , 103동 903호 （평촌동, 행복한마을아파트))",2019-07-01,2019-07-22,80000,"80,000"
2937,2019-000029-00,중구 당디로 95 （산성동）),2019-08-01,2021-08-31,100000,"100,000"
2939,2019-000031-00,"유성구 구즉로 25 , 306동 204호 （송강동, 송강그린아파트）)",2019-07-01,2019-07-22,80000,"80,000"
2940,2019-000032-00,"대덕구 신탄진동로 26-1, 예미지 A 303호 )",2019-09-03,2020-11-30,100000,"100,000"
2941,2019-000033-00,"대덕구 계족로663번길 34 , 305동 305호 （법동, 주공아파트）)",2019-08-01,2019-08-21,80000,"80,000"
2942,2019-000034-00,"대덕구 동심8길 2 , 208호 （대화동, 금성백조하우스）)",2019-08-01,2019-08-21,80000,"80,000"


In [22]:
# 남아있는 ) 제거
df_split["부과대상"] = df_split["부과대상"].str.replace(pat=r'\)', repl=r' ', regex=True)

In [23]:
df_split.head(2)

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세
0,2012-000002-00,"중구 동서대로11길 6,202호",2012-07-03,2012-07-18,40000,"40,000"
1,2014-000001-00,"서구 둔산로 223 , 6동 907호 （둔산동, 청솔아파트）",2014-08-12,2014-08-27,80000,"80,000"


정규식으로 안지워지는 괄호를 보니 `(` 가 아닌 두꺼운 괄호로 존재.. 

In [24]:
# 두꺼운 괄호 제거
df_split["부과대상"] = df_split["부과대상"].str.replace(pat=r'\（[^)]*\）', repl=r' ', regex=True)

In [25]:
df_split.head(2)

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세
0,2012-000002-00,"중구 동서대로11길 6,202호",2012-07-03,2012-07-18,40000,"40,000"
1,2014-000001-00,"서구 둔산로 223 , 6동 907호",2014-08-12,2014-08-27,80000,"80,000"


In [26]:
df_split_com = df_split[df_split['부과대상'].str.contains(',')]
df_split_ncom = df_split[~df_split['부과대상'].str.contains(',')]
print(df_split_com.shape)
print(df_split_ncom.shape)

(2419, 6)
(473, 6)


In [27]:
a = df_split_com['부과대상'].str.split(',', expand=True, n=1)

In [28]:
a

,0,1
0,중구 동서대로11길 6,202호
1,서구 둔산로 223,6동 907호
2,서구 둔산북로 160,102동 1401호
3,서구 유등로669번길 40,101호
4,서구 둔산북로 175,7동 1404호
...,...,...
3562,동구 대전로 422-13,401호
3563,동구 신기로 78,5층
3564,동구 백룡로57번길 138-7,심플A동 203호
3569,동구 계족로424번길 19,2층동


In [29]:
df_split_com['부과대상'] = a[0].copy()

In [30]:
# 병합
df_road = pd.concat([df_split_com, df_split_ncom], 
                    axis=0,
                    join='outer'
                    )

df_road.reset_index(drop=True)

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세
0,2012-000002-00,중구 동서대로11길 6,2012-07-03,2012-07-18,40000,"40,000"
1,2014-000001-00,서구 둔산로 223,2014-08-12,2014-08-27,80000,"80,000"
2,2014-000002-00,서구 둔산북로 160,2014-08-12,2014-08-27,80000,"80,000"
3,2014-000003-00,서구 유등로669번길 40,2014-08-12,2014-08-27,80000,"80,000"
4,2014-000004-00,서구 둔산북로 175,2014-08-12,2014-08-27,80000,"80,000"
...,...,...,...,...,...,...
2887,2014-000005-00,유성구 신성로61번안길 9,2014-03-14,2014-03-31,80000,"80,000"
2888,2014-000018-00,동구 창조1길 68,2014-07-21,2014-08-11,80000,"80,000"
2889,2014-000031-00,동구 중심2길 26,2014-08-21,2014-09-11,80000,"80,000"
2890,2015-000002-00,동구 용전동 12번지 17호 101호,2015-01-05,2015-02-02,50000,"50,000"


## kakao api 호출

In [31]:
df_kakao = df_road.copy()
df_kakao.reset_index(drop=True, inplace=True)

df_kakao['위도'] = np.nan
df_kakao['경도'] = np.nan
df_kakao['구'] = np.nan
df_kakao['행정동'] = np.nan

df_kakao.head(2)

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세,위도,경도,구,행정동
0,2012-000002-00,중구 동서대로11길 6,2012-07-03,2012-07-18,40000,"40,000",NaN,NaN,NaN,NaN
1,2014-000001-00,서구 둔산로 223,2014-08-12,2014-08-27,80000,"80,000",NaN,NaN,NaN,NaN


In [32]:
rest_api_key = "08d64ac0652c8f9e1c4ea45118910c53"

In [33]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [34]:
kakao = KakaoLocalAPI(rest_api_key)

In [35]:
add_list = df_kakao['부과대상']
add_list

0                중구 동서대로11길 6
1                 서구 둔산로 223 
2                서구 둔산북로 160 
3             서구 유등로669번길 40 
4                서구 둔산북로 175 
                ...          
2887         유성구 신성로61번안길 9  
2888             동구 창조1길 68  
2889             동구 중심2길 26  
2890    동구 용전동 12번지 17호 101호 
2891              동구 광명길 58  
Name: 부과대상, Length: 2892, dtype: object

In [36]:
# 호출 함수 확인
kakao.search_address(add_list[1]) 

{'documents': [{'address': {'address_name': '대전 서구 둔산동 1811',
    'b_code': '3017011200',
    'h_code': '3017066000',
    'main_address_no': '1811',
    'mountain_yn': 'N',
    'region_1depth_name': '대전',
    'region_2depth_name': '서구',
    'region_3depth_h_name': '둔산3동',
    'region_3depth_name': '둔산동',
    'sub_address_no': '',
    'x': '127.398129737198',
    'y': '36.3519333476051'},
   'address_name': '대전 서구 둔산로 223',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '대전 서구 둔산로 223',
    'building_name': '청솔아파트',
    'main_building_no': '223',
    'region_1depth_name': '대전',
    'region_2depth_name': '서구',
    'region_3depth_name': '둔산동',
    'road_name': '둔산로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.398129737198',
    'y': '36.3519333476051',
    'zone_no': '35246'},
   'x': '127.398129737198',
   'y': '36.3519333476051'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [37]:
for i in tqdm(range(len(add_list))):
    try :
        df_kakao['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        df_kakao['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass
    
    try:                
        df_kakao['구'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_2depth_name']
    except:
        pass        
    

    try:                
        df_kakao['행정동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_h_name']
    except:
        pass        
        
print(df_kakao.isnull().sum())

100%|██████████| 2892/2892 [11:39<00:00,  4.14it/s]

부과번호       0
부과대상       0
부과일자       0
최종납기일자     0
최초본세       0
최종본세       0
위도        87
경도        87
구         88
행정동       88
dtype: int64


2892개 중 87개 추출 실패

In [50]:
df_1st_fail = df_kakao[df_kakao['위도'].isnull()]
df_1st_save = df_kakao[~df_kakao['위도'].isnull()]

print(df_1st_fail.shape)
print(df_1st_save.shape)

(87, 10)
(2805, 10)


In [39]:
df_1st_save.head()

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세,위도,경도,구,행정동
1,2014-000001-00,서구 둔산로 223,2014-08-12,2014-08-27,80000,"80,000",36.3519333476051,127.398129737198,서구,둔산3동
2,2014-000002-00,서구 둔산북로 160,2014-08-12,2014-08-27,80000,"80,000",36.355609544089,127.390829984526,서구,둔산1동
3,2014-000003-00,서구 유등로669번길 40,2014-08-12,2014-08-27,80000,"80,000",36.3475861393594,127.39992074233,서구,탄방동
4,2014-000004-00,서구 둔산북로 175,2014-08-12,2014-08-27,80000,"80,000",36.355995286142,127.393399279418,서구,둔산1동
5,2014-000006-00,서구 월평동로 45,2014-08-12,2014-08-27,80000,"80,000",36.3586485129233,127.365742858289,서구,월평3동


## 위도 경도로 행정동 추출

In [40]:
df_dong = df_1st_save[df_1st_save['행정동'].isnull()]
df_dong 

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세,위도,경도,구,행정동
1900,2015-000039-00,대덕구 홍도로 113번길,2015-09-09,2015-11-02,100000,"100,000",36.3562068552128,127.425390549909,NaN,NaN


In [41]:
df_dong.reset_index(drop=True, inplace=True)
df_dong

,부과번호,부과대상,부과일자,최종납기일자,최초본세,최종본세,위도,경도,구,행정동
0,2015-000039-00,대덕구 홍도로 113번길,2015-09-09,2015-11-02,100000,"100,000",36.3562068552128,127.425390549909,NaN,NaN


In [42]:
# 위경도를 입력받는 kakao api 함수
def lat_lon_to_addr(lon,lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'.format(longitude=lon,latitude=lat)
    headers = {"Authorization": "KakaoAK " + rest_api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result
    return match_first

In [43]:
df_lati = df_dong['위도']
df_long = df_dong['경도']

In [44]:
df_long, df_lati

(0    127.425390549909
 Name: 경도, dtype: object,
 0    36.3562068552128
 Name: 위도, dtype: object)

In [45]:
# 함수 확인
lat_lon_to_addr(df_long[0], df_lati[0])['documents'][0]['region_3depth_name']

'중리동'

In [46]:
df_1st_save = df_1st_save.fillna('중리동')

In [47]:
df_1st_save.loc[1900]

부과번호        2015-000039-00
부과대상         대덕구 홍도로 113번길
부과일자            2015-09-09
최종납기일자          2015-11-02
최초본세                100000
최종본세               100,000
위도        36.3562068552128
경도        127.425390549909
구                      중리동
행정동                    중리동
Name: 1900, dtype: object

In [48]:
print(df_1st_save.isnull().sum())
print(df_1st_save.shape)

부과번호      0
부과대상      0
부과일자      0
최종납기일자    0
최초본세      0
최종본세      0
위도        0
경도        0
구         0
행정동       0
dtype: int64
(2805, 10)


총 2892개 중 2805개 전부 추출 완료

## 데이터프레임으로 저장

In [51]:
df_1st_save.to_csv(PATH + "/02_과태료/과태료_장병용_전처리_행정동 추출(2805).csv", index=False, encoding='cp949')